# Initial Setup

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

from utils.matchers import FeatureMatcher, MultipleInstanceMatcher
from utils.visualization import display_frames, display_ar_frames
from utils.utils import save_ar_video_f2r, save_ar_video_f2f, save_ar_video

#reloads external modules when they are changed
%load_ext autoreload
%autoreload 2

# Overview of the data

In [ ]:
video_filename = './Data/Multiple View.avi'

reference_frame = cv2.cvtColor(cv2.imread('Data/ReferenceFrame.png', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
reference_mask = cv2.imread('Data/ObjectMask.PNG', cv2.IMREAD_GRAYSCALE)

ar_layer = cv2.cvtColor(cv2.imread('Data/AugmentedLayer.PNG', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)[:,:640]
ar_mask = cv2.imread('Data/AugmentedLayerMask.PNG', cv2.IMREAD_GRAYSCALE)[:,:640]


## Reference frame

In [ ]:
w, h, dpi = 1000, 500, 100
fig, ax = plt.subplots(ncols=2, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(reference_frame)
ax[0].set_title('Reference frame')

ax[1].imshow(reference_mask, cmap='gray')
ax[1].set_title('Mask')

fig.tight_layout(pad=0.5)
plt.show()

## AR layer

In [ ]:
w, h, dpi = 1000, 500, 100
fig, ax = plt.subplots(ncols=2, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(ar_layer)
ax[0].set_title('AR layer')

ax[1].imshow(ar_mask, cmap='gray')
ax[1].set_title('Mask')

fig.tight_layout(pad=0.5)
plt.show()

## Video sequence

In [ ]:
display_frames(video_filename, starting_frame=0)

# Stitch AR layer onto reference frame

In [ ]:
ar_frame = reference_frame.copy()
ar_frame[ar_mask==255] = ar_layer[ar_mask==255]

w, h, dpi = 1200, 400, 100
fig, ax = plt.subplots(ncols=3, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(reference_frame)
ax[0].set_title('Reference frame')

ax[1].imshow(ar_layer)
ax[1].set_title('AR layer')

ax[2].imshow(ar_frame)
ax[2].set_title('Combination')

fig.tight_layout(pad=0.5)
plt.show()

# Do AR on the video
Overlay the AR layer onto the video frame.

At the beginning, the AR layer and a reference image onto which the AR layer should be stitched are needed.

Then the homography between the reference image and the video frame is computed, and it is then applied to the AR layer.

The homography between the reference image and the video frames is found using local invariant features, which are found using SIFT. The matches between the SIFT descriptors are then found using FLANN KDTree, and finally the homography is computed
on the resulting matches using RANSAC.

All this is implemented in the ``matchers.FeatureMatcher`` class.

Both the F2R (frame to reference) and F2F (frame to frame) have been tried. The results show that F2F produces a smoother result, but the AR layer drifts over time due to the accumulation of errors.

In F2F, the AR layer stays consistently in the right place, but the AR layer appears to be stuttering sometimes.
It was also tried to reset the reference frame after a certain number of frames.

In [ ]:
video_filename = './Data/Multiple View.avi'

reference_frame = cv2.cvtColor(cv2.imread('Data/ReferenceFrame.png', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
reference_mask = cv2.imread('Data/ObjectMask.PNG', cv2.IMREAD_GRAYSCALE)

ar_layer = cv2.cvtColor(cv2.imread('Data/AugmentedLayer.PNG', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)[:,:640]
ar_mask = cv2.imread('Data/AugmentedLayerMask.PNG', cv2.IMREAD_GRAYSCALE)[:,:640]


F2R

In [ ]:
save_ar_video_f2r(video_filename,
                  'out_f2r.avi',
                  ar_layer=ar_layer,
                  ar_mask=ar_mask,
                  reference_image=reference_frame,
                  reference_mask=reference_mask)

F2F

In [ ]:
save_ar_video_f2f(video_filename,
                  'out_f2f.avi',
                  ar_layer=ar_layer,
                  ar_mask=ar_mask,
                  reference_image=reference_frame,
                  reference_mask=reference_mask)

F2F with reset

In [ ]:
r = 30
save_ar_video(video_filename,
              f'out_h{r}.avi',
              ar_layer=ar_layer,
              ar_mask=ar_mask,
              reference_image=reference_frame,
              reference_mask=reference_mask,
              drift_correction_step=r)